## Sig53 Dataset

last updated: 2023-07-29

TODO: This notebook is just a placeholder for now.

### Modulation Family Background

TODO: Describe and plot the modulation families present in Sig53. Should resemble Figure 3 from [Large Scale Radio Frequency Signal Classification](https://arxiv.org/pdf/2207.09918.pdf).

In [ ]:
#TODO: Replace this code block
import matplotlib.pyplot as plt
import numpy as np

num_samples = 4096
x = np.exp(2j * np.pi * 2.0 / num_samples * np.arange(num_samples))

plt.figure()
plt.plot(x.real, c='b')
plt.plot(x.imag, c='r')
plt.title('Test plot')
plt.xlabel('samples')
plt.ylabel('amplitude')
plt.show()

### Instantiate the Sig53 Dataset

TODO: Explain details of the Sig53 dataset and show code on how to instantiate the datasets.

In [ ]:
#TODO: Replace this code block

from torchsig.transforms import AddNoise

t = AddNoise(noise_power_db=-20)

y = t(x)

plt.figure()
plt.plot(y.real, c='b')
plt.plot(y.imag, c='r')
plt.title('Test plot')
plt.xlabel('samples')
plt.ylabel('amplitude')
plt.show()